In [1]:
## Install libraries
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.config("spark.driver.memory", "8g").getOrCreate()
pathData = "D:\\Đại Học CNTT\\Data engineer\\DE-COURSE\\Homework\\ETL Pineline\\data\\20220401.json"

In [3]:
df = spark.read.json(pathData)
df.show(10,truncate=False)

+--------------------+-------+------+--------------------------------------+-----+
|_id                 |_index |_score|_source                               |_type|
+--------------------+-------+------+--------------------------------------+-----+
|AX_momhia1FFivsGrn9o|history|0     |{KPLUS, HNH579912, 0C96E62FC55C, 254} |kplus|
|AX_momhca1FFivsGrnvg|history|0     |{KPLUS, HUFD40665, CCEDDC333614, 1457}|kplus|
|AX_momhaa1FFivsGrnny|history|0     |{KPLUS, HNH572635, B068E6A1C5F6, 2318}|kplus|
|AX_momhca1FFivsGrnvv|history|0     |{KPLUS, HND141717, 08674EE8D2C2, 1452}|kplus|
|AX_momhia1FFivsGrn98|history|0     |{KPLUS, HNH743103, 402343C25D7D, 251} |kplus|
|AX_momg9a1FFivsGrnkS|history|0     |{KPLUS, HNH893773, B84DEE76D3B8, 924} |kplus|
|AX_momhca1FFivsGrnwA|history|0     |{KPLUS, HND083642, B84DEE849A0F, 1444}|kplus|
|AX_momhfa1FFivsGrn2u|history|0     |{KPLUS, DNFD74404, 90324BB44C39, 691} |kplus|
|AX_momhca1FFivsGrnwP|history|0     |{KPLUS, DTFD21200, B84DEED27709, 1436}|kplus|
|AX_

In [4]:
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- _index: string (nullable = true)
 |-- _score: long (nullable = true)
 |-- _source: struct (nullable = true)
 |    |-- AppName: string (nullable = true)
 |    |-- Contract: string (nullable = true)
 |    |-- Mac: string (nullable = true)
 |    |-- TotalDuration: long (nullable = true)
 |-- _type: string (nullable = true)



In [5]:
df = df.select('_source.AppName', '_source.Contract', '_source.Mac', '_source.TotalDuration')

In [6]:
df.show(5)

+-------+---------+------------+-------------+
|AppName| Contract|         Mac|TotalDuration|
+-------+---------+------------+-------------+
|  KPLUS|HNH579912|0C96E62FC55C|          254|
|  KPLUS|HUFD40665|CCEDDC333614|         1457|
|  KPLUS|HNH572635|B068E6A1C5F6|         2318|
|  KPLUS|HND141717|08674EE8D2C2|         1452|
|  KPLUS|HNH743103|402343C25D7D|          251|
+-------+---------+------------+-------------+
only showing top 5 rows



In [7]:
df = df.withColumn('Type',
        when((col('AppName') == 'CHANNEL') | (col('AppName') =='DSHD')| (col('AppName') =='KPLUS')| (col('AppName') =='KPlus'), 'Truyền Hình')
        .when((col('AppName') == 'VOD') | (col('AppName') =='FIMS_RES')| (col('AppName') =='BHD_RES')| 
                (col('AppName') =='VOD_RES')| (col('AppName') =='FIMS')| (col('AppName') =='BHD')| (col('AppName') =='DANET'), 'Phim Truyện')
        .when((col('AppName') == 'RELAX'), 'Giải Trí')
        .when((col('AppName') == 'CHILD'), 'Thiếu Nhi')
        .when((col('AppName') == 'SPORT'), 'Thể Thao')
        .otherwise('Error'))


In [9]:
data = df.withColumn("Date", lit("2022-04-01"))
data.show(5)

+-------+---------+------------+-------------+-----------+----------+
|AppName| Contract|         Mac|TotalDuration|       Type|      Date|
+-------+---------+------------+-------------+-----------+----------+
|  KPLUS|HNH579912|0C96E62FC55C|          254|Truyền Hình|2022-04-01|
|  KPLUS|HUFD40665|CCEDDC333614|         1457|Truyền Hình|2022-04-01|
|  KPLUS|HNH572635|B068E6A1C5F6|         2318|Truyền Hình|2022-04-01|
|  KPLUS|HND141717|08674EE8D2C2|         1452|Truyền Hình|2022-04-01|
|  KPLUS|HNH743103|402343C25D7D|          251|Truyền Hình|2022-04-01|
+-------+---------+------------+-------------+-----------+----------+
only showing top 5 rows



In [13]:
data.groupBy(col("AppName")).count().show()

+-------+-------+
|AppName|  count|
+-------+-------+
|  KPLUS|  20158|
|  RELAX|   6445|
|  CHILD|  33723|
|CHANNEL|1502855|
|    VOD|  88033|
|   FIMS|   1007|
|  SPORT|   2298|
+-------+-------+



In [10]:
data.groupBy("Date","Contract","Type").agg((sum('TotalDuration').alias("TotalDuration"))).show(5)

+----------+---------+-----------+-------------+
|      Date| Contract|       Type|TotalDuration|
+----------+---------+-----------+-------------+
|2022-04-01|NBFD22476|Truyền Hình|        18948|
|2022-04-01|HPD067082|Truyền Hình|        83543|
|2022-04-01|BGFD62840|Truyền Hình|        18892|
|2022-04-01|NBFD14577|Truyền Hình|        18852|
|2022-04-01|SGH799584|Truyền Hình|        25484|
+----------+---------+-----------+-------------+
only showing top 5 rows



In [12]:
data.groupBy('Date','Contract').pivot('Type').sum('TotalDuration').show(5)

+----------+---------+--------+-----------+---------+--------+-----------+
|      Date| Contract|Giải Trí|Phim Truyện|Thiếu Nhi|Thể Thao|Truyền Hình|
+----------+---------+--------+-----------+---------+--------+-----------+
|2022-04-01|NTFD29164|    NULL|       8344|     NULL|    NULL|       5772|
|2022-04-01|HTFD18781|    NULL|       NULL|     NULL|    NULL|       5975|
|2022-04-01|HPFD67165|    NULL|       NULL|     NULL|    NULL|      60623|
|2022-04-01|TBFD31164|    NULL|       NULL|     NULL|    NULL|      17426|
|2022-04-01|NDFD21780|    NULL|       NULL|     NULL|    NULL|       5666|
+----------+---------+--------+-----------+---------+--------+-----------+
only showing top 5 rows

